<a href="https://colab.research.google.com/github/DaiZack/CAhousing/blob/master/findDuplicatedAricle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: load your data

In [ ]:
import pandas as pd

filename ='news combined - round 2.xlsx'
outputfile = 'WithOrign_'+filename

df = pd.read_excel('news combined - round 2.xlsx')
df.head()

,title,date,author,wordscount,content,Journal
0,MarketWatch: Low Mortgage Rates Will Cost You,2009-02-22,By Amy Hoak,910,"Mortgage rates are low, but getting a home loa...",Wall Street Journal
1,Marketwatch; Low Mortgage Rates Will Cost You,2009-02-22,By Amy Hoak,910,"Mortgage rates are low, but getting a home loa...",Wall Street Journal
2,Green dreams get muddy for car maker --- Norwe...,2009-01-27,By Leila Abboud,1076,"In the race to get electric cars to market, th...",Wall Street Journal
3,Faded Green: A Car Maker's Woes; Start-Up Lure...,2009-01-26,By Leila Abboud,1066,"In the race to get electric cars to market, th...",Wall Street Journal
4,Corporate News: Faded Green: A Car Maker's Woe...,2009-01-26,By Leila Abboud,1084,"In the race to get electric cars to market, th...",Wall Street Journal


# step 2 use sklearn counterVectorizer to convert the text into vector(matrix)

hint: use binary=True, so we only count the word appreance as 1 if exist.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True)
vect = vectorizer.fit_transform(df['content'])

# step 3 use matric multiplication  

In [ ]:
import numpy as np
similarity = vect @ vect.T

#### let a look about the similarity
each row and the column present one article, the number means how many words same words are used.

note that the diagonal is the articel to itself which should alwarys be the highest score and that equals to how many words in the article.

In [ ]:
similarity.toarray()

array([[371, 360, 121, ...,  96, 135, 131],
       [360, 363, 111, ...,  96, 135, 131],
       [121, 111, 503, ..., 115, 188, 160],
       ...,
       [ 96,  96, 115, ..., 507, 194, 172],
       [135, 135, 188, ..., 194, 947, 296],
       [131, 131, 160, ..., 172, 296, 870]], dtype=int64)

In the matric about, cell (1,1) = 371 means article 1 has 371 words.

cell (1,2) = 360 means articel 2 has 360 common words with articel 1. which means they are 97% similiar

lets print them out.

In [ ]:
df.loc[0,'content']

'Mortgage rates are low, but getting a home loan is going to cost you.\nNew rules by \nFreddie Mac\n and \nFannie Mae\n are upping the fees for borrowers with less than perfect credit, those in the mortgage industry say. Other increased costs reflect the uncertainty in the mortgage market as lenders try to reduce their risk and anticipate rates.\n"It\'s an interesting time, in that mortgage rates are historically low," says Amy Bohutinsky, vice president of communications for Zillow.com, a real-estate Web site. "But at the same time, while rates are low, lending standards are still really tight. What that means is that people who qualify for these really good rates ... fall under a strict set of guidelines."\nEven borrowers with decent credit aren\'t immune to higher fees and mortgage costs. In general, to get the low rates that make the headlines, borrowers also are often paying more points, or prepaid interest, that bring the mortgage rate down.\nIf you look at where mortgage pricing

In [ ]:
df.loc[1,'content']

'Mortgage rates are low, but getting a home loan is going to cost you.\nNew rules by \nFreddie Mac\n and \nFannie Mae\n are upping the fees for borrowers with less than perfect credit, those in the mortgage industry say. Other increased costs reflect the uncertainty in the mortgage market as lenders try to reduce their risk and anticipate rates.\n"It\'s an interesting time, in that mortgage rates are historically low," says Amy Bohutinsky, vice president of communications for \nZillow.com\n, a real-estate Web site. "But at the same time, while rates are low, lending standards are still really tight. What that means is that people who qualify for these really good rates … fall under a strict set of guidelines."\nEven borrowers with decent credit aren\'t immune to higher fees and mortgage costs. In general, to get the low rates that make the headlines, borrowers also are often paying more points, or prepaid interest, that bring the mortgage rate down.\nPricing Changes\nIf you look at whe

we can see they are almost identical

# step 4 calculate the similarity %

In [ ]:
articleWords = similarity.max(axis=1).toarray().flatten()
articleWords

array([371, 363, 503, ..., 507, 947, 870])

In [ ]:
similarityPercent = similarity/articleWords
similarityPercent

matrix([[1.        , 0.99173554, 0.24055666, ..., 0.18934911, 0.14255544,
         0.15057471],
        [0.9703504 , 1.        , 0.22067594, ..., 0.18934911, 0.14255544,
         0.15057471],
        [0.32614555, 0.30578512, 1.        , ..., 0.22682446, 0.19852165,
         0.18390805],
        ...,
        [0.25876011, 0.26446281, 0.22862823, ..., 1.        , 0.20485744,
         0.19770115],
        [0.3638814 , 0.37190083, 0.37375746, ..., 0.382643  , 1.        ,
         0.34022989],
        [0.35309973, 0.36088154, 0.31809145, ..., 0.33925049, 0.312566  ,
         1.        ]])

you can see the diagonal values are all 1

note that cell (1,2) = 0.9917 and cell (2,1) = 0.97035. the reason for the diffenece is the denominators are different:

cell (1,2) = 360/363

cell (2,1) = 360/371

# step 5 add originalID column

In [ ]:
threshold = 0.9 # lets say if the % higher ore equal to 90% are duplicates


let do a simple test

In [ ]:
np.where(similarityPercent[0] >= threshold)[-1]

array([0, 1])

the where shows us the index of the articles that are similar

loop the entire list, for each time find the duplicated the articel and keep the original one.

In [ ]:
keep = set()
replacedict = {}
for i,line in enumerate(similarityPercent):
  dupeCluster = np.where(line>threshold)[-1]
  if set(dupeCluster).intersection(set(keep)):
    replacedict[i] = list(set(dupeCluster).intersection(set(keep)))[0]
  else:
    keep.add(i)
    replacedict[i] = None


In [ ]:
replacedf = pd.DataFrame(replacedict.items(), columns=['id','originid']).set_index('id')

In [ ]:
df = df.merge(replacedf, left_index=True, right_index=True,how='left').reset_index().rename(columns={df.index.name:'id'})

In [ ]:
df.to_excel(outputfile, index=False)

In [ ]:
from google.colab import files
files.download(outputfile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>